In [8]:
import pandas as pd
import spacy
from typing import Dict, Tuple


nlp = spacy.load("en_core_web_sm")


file_path = '/content/electronica_exhibitors_with_contacts.csv'
data = pd.read_csv(file_path)


data_cleaned = data[['Company Name', 'Address', 'Description', 'Website', 'Email', 'Contact Number']]
data_cleaned['Country'] = data_cleaned['Address'].str.extract(r',\s*([^,]+)$')


data_cleaned['Description'] = data_cleaned['Description'].str.lower()


synonyms = {
    "semiconductors": ["semiconductors", "chips", "processors"],
    "microcontrollers": ["microcontrollers", "embedded controllers", "MCUs"],
    "all": ["all"]
}

def expand_query(query: str) -> str:
    """
    Expand user query with synonyms for better matching.
    """
    for key, words in synonyms.items():
        if any(word in query for word in words):
            return key
    return query

def parse_query(user_input: str, context: Dict[str, str]) -> Tuple[str, str, str]:
    """
    Parse user input to extract intent, entity, and location using spaCy.
    Leverages context for follow-up queries.
    """
    doc = nlp(user_input)
    intent, entity, location = None, None, None


    if "how many" in user_input or "count" in user_input:
        intent = "count"
    elif "list" in user_input or "manufacture" in user_input:
        intent = "list"
    elif "contact" in user_input:
        intent = "contact"

    for token in doc:
        if token.ent_type_ == "GPE":
            location = token.text.lower()
        if token.text.lower() in synonyms.get("all", []) + synonyms.get("semiconductors", []) + synonyms.get("microcontrollers", []):
            entity = expand_query(token.text.lower())


    if not location and "location" in context:
        location = context["location"]
    if not entity and "entity" in context:
        entity = context["entity"]

    return intent, entity, location

class Chatbot:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.context = {}

    def respond(self, user_input: str) -> str:
        """
        Generate responses based on user queries and dataset.
        """
        intent, entity, location = parse_query(user_input, self.context)


        if location:
            self.context["location"] = location
        if entity:
            self.context["entity"] = entity

        if intent == "count":
            return self.handle_count(entity, location)
        elif intent == "list":
            return self.handle_list(entity, location)
        elif intent == "contact":
            return self.handle_contact(location)
        else:
            return "I'm sorry, I couldn't understand that. Try asking about company counts, lists, or contact details."

    def handle_count(self, entity: str, location: str) -> str:
        """
        Handle count queries.
        """
        if not entity:
            return "Please specify an industry (e.g., Semiconductors or Microcontrollers)."
        results = self.data[self.data['Description'].str.contains(entity, case=False, na=False)]
        return f"There are {len(results)} companies listed under the '{entity}' industry."

    def handle_list(self, entity: str, location: str) -> str:
        """
        Handle list queries.
        """
        if not entity:
            return "Please specify what you'd like to list (e.g., Semiconductors or Microcontrollers)."
        results = self.data[self.data['Description'].str.contains(entity, case=False, na=False)]
        if not results.empty:
            return f"Found {len(results)} companies manufacturing '{entity}'. Here are some:\n" + \
                   results[['Company Name', 'Description']].head(5).to_string(index=False)
        else:
            return f"No companies found manufacturing '{entity}'."

    def handle_contact(self, location: str) -> str:
        """
        Handle contact queries.
        """
        if not location:
            return "Please specify a location to provide contact details."
        results = self.data[self.data['Country'].str.contains(location, case=False, na=False)]
        if not results.empty:
            return f"Contact details for companies in {location}:\n" + \
                   results[['Company Name', 'Email', 'Contact Number']].head(5).to_string(index=False)
        else:
            return f"No contact details found for companies in {location}."


chatbot = Chatbot(data_cleaned)


print("Welcome to the Company Chatbot! Ask your queries or type 'exit' to quit.")
while True:
    user_query = input("\nYou: ")
    if user_query.lower() == 'exit':
        print("Goodbye!")
        break
    response = chatbot.respond(user_query)
    print("\nChatbot:", response)


Welcome to the Company Chatbot! Ask your queries or type 'exit' to quit.

You: List all companies that manufacture 'Microcontrollers'.

Chatbot: No companies found manufacturing 'microcontrollers'.

You: Provide the contact details of companies located in 'Germany'.

Chatbot: Contact details for companies in germany:
                              Company Name                       Email    Contact Number
              2E mechatronic GmbH & Co. KG n-philipp@2e-mechatronic.de 0049 7021 9301 23
                       3M Deutschland GmbH              No email found     0049 2131 140
                        3PEAK Incorporated     business_EMEA@3peak.com  0086 21 51090810
Karl-Heinz Mauz GmbH EKULIT Elektrotechnik              info@ekulit.de  0049 711 3414023
              2E mechatronic GmbH & Co. KG n-philipp@2e-mechatronic.de 0049 7021 9301 23

You: exit
Goodbye!
